In [2]:
import numpy as np

In [1]:
def linear(x):
    return  x

In [5]:
def sigmoidal(x):
    return 1/(1 + np.exp(-x))

In [6]:
print(linear(3))
print(sigmoidal(10))

3
0.9999546021312976


In [11]:
class MLP:
    def __init__(self, layers, weights, activation_function):
        self.layers = layers
        self.weights = weights

        if activation_function == "linear":
            self.hidden_activation_function = linear
        elif activation_function == "sigmoidal":
            self.hidden_activation_function = sigmoidal
        else:
            print("Wrong activation function specified")
            return 
        
        # initialization
        for layer in layers:
            print(layer)


        self.output_activation_function = linear
    
    # @ - mnożenie macierzy

    # def forward(self):


input_size = 5

NN1 = MLP([input_size,5], [], 'sigmoidal')

NN2 = MLP([input_size, 10])

NN3 = MLP([input_size, ])


5
10
10
